In [230]:
using Plots, ComplexPhasePortrait, ApproxFun, SingularIntegralEquations, DifferentialEquations
gr();

# M3M6: Methods of Mathematical Physics

$$
\def\dashint{{\int\!\!\!\!\!\!-\,}}
\def\infdashint{\dashint_{\!\!\!-\infty}^{\,\infty}}
\def\D{\,{\rm d}}
\def\dx{\D x}
\def\dt{\D t}
\def\C{{\mathbb C}}
\def\CC{{\cal C}}
\def\HH{{\cal H}}
\def\I{{\rm i}}
\def\qqfor{\qquad\hbox{for}\qquad}
$$

Dr. Sheehan Olver
<br>
s.olver@imperial.ac.uk

Office Hours: 3-4pm Mondays, Huxley 6M40
<br>
Website: https://github.com/dlfivefifty/M3M6LectureNotes

# Lecture 12: Electric charges in a potential well

1. Additive Riemann–Hilbert problem $\phi_+(x) + \phi_-(x) = f(x)$
   - General solution on $[-1,1]$ built _from_ a Cauchy transform
   - General solution on $[-1,1]$ expressed _as_ a Cauchy transform
   - Inverting the Hilbert transform
2. Application: Electric charges in a potential well
   - One charge in a well
   - Two charges in a well
   - $N$ charges in a well
   - Limiting distribution

## Additive Riemann–Hilbert problem

We now consider an additive Riemann–Hilbert problem
$$
\phi_+(x) + \phi_-(x) = f(x) \qquad \phi(\infty) = 0
$$
and take $\Gamma = [-1,1]$. 


### General solution on $[-1,1]$ built _from_ a Cauchy transform


Here we propose the general solution to an additive Riemann–Hilbert problem:

**Theorem (Cauchy transform solution to Additive Riemann–Hilbert problem)** Suppose $\phi(z)$ analytic in $\C \backslash [-1,1]$ satisfies

1. $\phi_+(x) + \phi_-(x) = f(x) $ for $-1 < x < 1$, where $f$ is smooth on $[-1,1]$.
2. $\phi(z)$ has weaker than pole singularities,
3. $\phi(\infty) = 0$.

Then, for some constant $C$, 
$$
 \phi(z) = {\I \over \sqrt{z-1}\sqrt{z+1}} \CC_{[-1,1]} \left[{ f(\diamond)  \sqrt{1-\diamond^2} }\right](z)  + {C \over \sqrt{z-1} \sqrt{z+1}}
$$


In [209]:
x = Fun()
f = exp(x)
v = f*sqrt(1-x^2)

C = randn() # doesn't matter

φ = z -> im*cauchy(v, z)/(sqrt(z-1)sqrt(1+z)) + C/(sqrt(z-1)sqrt(z+1))

@show φ(1.0E8)
@show f(0.1)
@show φ(0.1+0.0im) + φ(0.1-0.0im);

phaseplot(φ, (-3,3), (-2,2))

φ(1.0e8) = -9.823815705608112e-9 + 0.0im
f(0.1) = 1.1051709180756475
φ(0.1 + 0.0im) + φ(0.1 - 0.0im) = 1.1051709180756475 + 0.0im


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2

On other intervals $(a,b)$, we get the same forms of solution:

$$
 \phi(z) = {\I \over \sqrt{z-b}\sqrt{z-a}} \CC_{[a,b]} \left[{ f(\diamond)  \sqrt{b-x}\sqrt{x-a} }\right](z)  + {C \over \sqrt{z-b} \sqrt{z-a}}
$$

### General solution on $[-1,1]$ _expressed as_ a Cauchy transform

Note the expression above is _not_ a Cauchy transform of a function. We can determine a Cauchy transform expression by taking the difference: on $[-1,1]$ we have
$$
    u(x) = \phi_+(x) - \phi_-(x) = {1 \over \sqrt{1 - x^2}}\left(        \CC_{[-1,1]}^+ + \CC_{[-1,1]}^- \right) \left[{ f(\diamond)  \sqrt{1-\diamond^2} }\right](x) - {2C \I \over \sqrt{1-x^2}}
$$
and by (**Plemelj II**), we guarantee that 
$$
\phi(z) = \CC_{[-1,1]} u(z)
$$


In the special case where we can calculate $\CC \left[{ f(\diamond)  \sqrt{1-\diamond^2} }\right](z)$ exactly, we can work out the precise formula for $u(x)$.


**Example** Consider solving

1. $\phi_+(x) + \phi_-(x) = x $ for $-b < x < b$
2. $\phi(z)$ has weaker than pole singularities,
3. $\phi(\infty) = 0$.

Note that 
$$
    \kappa(z) = {z \sqrt{z-b} \sqrt{z+b} - z^2 \over 2 \I} 
$$
satisfies
$$
\kappa^+(x) - \kappa^-(x) = x \sqrt{b^2 - x^2}
$$

In [218]:
b = 2
κ = z -> (z*sqrt(z-b)*sqrt(z+b) - z^2)/(2im)  +D
@show κ(1E5)
κ(0.1+0.0im)- κ(0.1-0.0im),   0.1sqrt(b^2-0.1^2)

κ(100000.0) = -0.0 + 1.0im


(0.1997498435543818 + 0.0im, 0.1997498435543818)

Hence every solution has the form
\begin{align*}
\phi(z) = {1 \over \sqrt{z-b} \sqrt{z+b}} \left({z \sqrt{z-b} \sqrt{z+b} - z^2\over 2 }  + D\right) + {C \over \sqrt{z-b} \sqrt{z+b}} \\
=  {z  \over 2 } - {z^2 \over 2  \sqrt{z-b} \sqrt{z+b}} + {C \over \sqrt{z-b} \sqrt{z+b}}
\end{align*}

In [172]:
C = randn()

φ = z -> z/(2) - z^2/(2*sqrt(z-b)*sqrt(z+b)) + C/(sqrt(z-b)*sqrt(z+b))
φ(0.1+0.0im) + φ(0.1-0.0im)

0.1 + 0.0im

Inspection reveals that
$$
\phi_+(x) - \phi_-(x) = \I {x^2 - C \over  \sqrt{b^2-x^2}}
$$

In [175]:
u = im*(x^2 - C)/sqrt(b^2 - x^2)

cauchy(u, 0.1+0.0im) + cauchy(u, 0.1-0.0im)

0.1 + 0.0im

$C$ is a free paremeter, hence, similar to ODEs, we can add a boundary condition. In other words we can ask for the solution satisfying, for example $u(0) = 0$. This is precisely when $C = 0$ and $u(x) = \I x^2 / \sqrt{b^2-x^2}$.

Often we ask for the solution that is bounded at the left/right endpoint. Because of the symmetry in the problem (since $x$ is an odd function), we can actually get a solution that is bounded at both $\pm b$ by choosing $C =  b^2$:
$$
\phi_+(x) - \phi_-(x) = -\I \sqrt{b^2 - x^2}
$$

In [195]:
b = 2.3
x = Fun(-b .. b)
u = -im*sqrt(b^2 - x^2)

cauchy(u, 0.1+0.0im) + cauchy(u, 0.1-0.0im)

0.1 + 0.0im

In other words,
$$
\phi(z) = - \I \CC_{[-b,b]} \sqrt{b^2 - \diamond^2}(z)
$$
is a bounded solution to the additive Riemann–Hilbert problem.

### Inverting the Hilbert transform

We can use this along with Plemelj's lemma to invert the Hilbert transform. That is, we want to find $u(x)$ such that
$$
\HH u(x) = f(x)
$$
but we know from (**Plemelj III**) that $\CC u(x)$ satisfies $\CC^+ u(x) + \CC^- u(x) = -\I \HH u(x)$. And from (**Plemelj II**) we know that $u(x) = \CC^+ u(x) - \CC^- u(x)$. Thus we can recover such a $u$ from $\phi$ satisfying $\phi_+(x) + \phi_-(x) = f(x)$ via
$$
    u(x) = -\I (\phi_+(x) - \phi_-(x))
$$
so that
$$
    \CC u(z) =  -\I \phi(z)
$$
and
$$
    \HH u(x) = \I (\CC^+ + \CC^-) u(x) = \phi_+(x) + \phi_-(x) = f(x).
    $$
    
** Example** 

Consider $\HH_{[-b,b]}  v(x) = x$. We have 
$$
- \I \HH_{[-b,b]} \sqrt{b^2 - \diamond^2}(x) = \CC_{[-b,b]}^+ \sqrt{b^2 - \diamond^2}(z) + \CC_{[-b,b]}^- \sqrt{b^2 - \diamond^2}(z) = \I (\phi^+(x) + \phi^-(x) ) = \I x
$$
which shows that the solution is
$$
v(x)  = - \sqrt{b^2 - \diamond^2}
$$

In [199]:
v = -sqrt(b^2 - x^2)

plot(v; label = "v")
plot!(hilbert(v); ylims=(-3,3), label="H[v]")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 v 
 
 
 
 H[v]



**Remark** As far as I can tell, if we are interested in deriving exact solutions, solving the equation $\phi_+(x) + \phi_-(x) = f(x)$ directly is just as easy as calculating the Cauchy transform. However, when there are multiple intervals the Cauchy transform has the benefit of decoupling the problems.

Unlike subtractive Riemann–Hilbert problems,  if $\Gamma$ is in the domain of analyticity of a non-zero analytic function $g(z)$, we do not have $\phi_+(x) + \phi_-(x) = g(x)$. 


## Application: electric charges in a potential well


We consider the problem of studying the dynamics of many electric charges in a potential well.  We restrict our attention to 1D: picture an infinitely long wire with carges on it. We will see that as the number of charges becomes large, we can determine the limitting distribution using an additive Riemann–Hilbert problem.

### One charge in a potential well 

Consider a point charge in a well $V(x) = x^2 / 2$, initially located at $\lambda^0$. The dynamics of the point charge are governed by
$$
{ \D \lambda \over \dt} = -V'(\lambda) = -  \lambda
$$
that is: if we are positive we move left and if we are negative we move right. Here is a movie:



In [231]:
V = x -> x^2/2
Vp = x -> x

λ⁰ = 2.3  # initial location
prob = ODEProblem((t,λ) -> -Vp(λ), λ⁰, (0.0, 10.0))
λ = solve(prob; reltol=1E-6);

In [232]:
@gif for t=0.0:0.05:7.0
    plot(V, linspace(-5, 5, 100); label="potential", title="t = $t")
    scatter!([λ(t)] ,[0.0]; label="charge")
end

INFO: Saved animation to /Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif


Plots.AnimatedGif("/Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif")

In the limit the charge reaches an equilibrium: it no longer varies in time. I.e., it reaches a point where ${\D\lambda \over \dt} = 0$, which is equivalent to solving
$$
 0 = - V'(\lambda)  = - \lambda
$$
in other words, the minimum of the well, in this case $\lambda = 0$.

### Two charges in a potential well 

Suppose there are now two charges, $\lambda_1$ and $\lambda_2$.  The effect on the first charge $\lambda_1$ is to repulse away from $\lambda_2$ via via:
$$
{\D \lambda_1 \over \D t} = {1 \over \lambda_1 -\lambda_2}
$$
Similarly, the effect on $\lambda_2$ is
$$
{\D \lambda_2 \over \D t} = {1 \over \lambda_2 -\lambda_1}
$$
Unrestricted, the two potentials will repulse off to infinity:

In [233]:
λ⁰ = [1.2,2.3]  # initial location
prob = ODEProblem((t,λ) -> [1/(λ[1] - λ[2]), 1/(λ[2] - λ[1])], λ⁰, (0.0, 10.0))
λ = solve(prob; reltol=1E-6);

In [234]:
@gif for t=0.0:0.05:7.0
    scatter(λ(t) ,zeros(2); label="charges", xlims=(-5,5), title="t = $t")
end

INFO: Saved animation to /Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif


Plots.AnimatedGif("/Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif")

Adding in a potential well and we get an equilbrium again:
\begin{align*}
{\D \lambda_1 \over \D t} = {1 \over \lambda_1 -\lambda_2} - V'(\lambda_1) \\
{\D \lambda_2 \over \D t} = {1 \over \lambda_2 -\lambda_1} - V'(\lambda_2)
\end{align*}


In [224]:
λ₀ = [1.2,2.3]  # initial location
prob = ODEProblem((t,λ) -> [1/(λ[1] - λ[2]) - Vp(λ[1]), 1/(λ[2] - λ[1]) - Vp(λ[2])], λ₀, (0.0, 10.0))
λ = solve(prob; reltol=1E-6);

In [225]:
@gif for t=0.0:0.05:7.0
    plot(V, linspace(-5, 5, 100); label="potential", title="t = $t")
    scatter!(λ(t) ,zeros(2); label="charges", xlims=(-5,5), title="t = $t")
end

INFO: Saved animation to /Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif


Plots.AnimatedGif("/Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif")

The limiting distribution is given by
\begin{align*}
0 = {1 \over \lambda_1 -\lambda_2} - V'(\lambda_1) \\
0 = {1 \over \lambda_2 -\lambda_1} - V'(\lambda_2) 
\end{align*}

### $N$ charges in a potential well

Each charge repulses every other charge, so we end up needing to sum over them all:
\begin{align*}
{\D \lambda_k \over \D t} = \sum_{j=1 \atop j \neq k}^N {1 \over \lambda_k -\lambda_j} - V'(\lambda_k) \\
\end{align*}



In [235]:
N = 100
λ⁰ = randn(N)  # initial location
prob = ODEProblem((t,λ) -> Float64[sum(1./(λ[k] .- λ[[1:k-1;k+1:end]])) - Vp(λ[k]) for k=1:N], λ⁰, (0.0, 10.0))
λ = solve(prob; reltol=1E-6);

In [236]:
@gif for t=0.0:0.05:7.0
    plot(V, linspace(-5, 5, 100); label="potential", title="t = $t")
    scatter!(λ(t) ,zeros(N); label="charges", xlims=(-5,5), title="t = $t")
end

INFO: Saved animation to /Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif


Plots.AnimatedGif("/Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif")

As the number of charges becomes large, they spread off to infinity. In the case of $V(x) = x^2$, we can renormalize by  dividing by $N$ so they stay bounded:
$\mu_k = {\lambda_k \over \sqrt N}$

In [237]:
@gif for t=0.0:0.05:7.0
    scatter(λ(t)/sqrt(N) ,zeros(N); label="charges", xlims=(-2,2), title="t = $t")
end

INFO: Saved animation to /Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif


Plots.AnimatedGif("/Users/solver/Documents/Coding/Lectures/M3M6 Methods of Mathematical Physics/tmp.gif")

This begs questions: why does it balance out at $\pm \sqrt 2$? Why does it have a nice histogram precisely like ${\sqrt{2-x^2} \over \pi}$

In [244]:
histogram(λ(10.0)/sqrt(N); nbins=20, normalize=true, label="histogram of charges")
plot!(x -> sqrt(2-x^2)/(π), linspace(eps()-sqrt(2.0),sqrt(2)-eps(),100), label="semicircle")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 histogram of charges 
 
 
 
 semicircle

### Equilibrium distribution

Plugging in $\lambda_k = \sqrt N \mu_k$, we get a dynamical system for $\mu_k$:
$$
 {\D  \mu_k \over \D t} = {1 \over N} \sum_{j=1 \atop j \neq k}^N {1 \over \mu_k -\mu_j} -  \mu_k
$$
(The choice of scaling like $\sqrt N$ was dictated by $V'(x)$, if $V(x) = x^4$ it would be $N^{1/4}$.) Thus the limit of the charges is given by
$$
0 = {1 \over N} \sum_{j=1 \atop j \neq k}^N {1 \over \mu_k -\mu_j} -  \mu_k 
$$



It is convenient to represent the point charges by Dirac delta functions:
$$
    w_N(x) = {1 \over N} \sum_{k=1}^N \delta_{\mu_k}(x)
$$
normalized so that $\int w_N(x) \dx = 1$, so that
$$
    {1 \over N} \sum_{k=1}^N {1 \over  x -\mu_j} = \int_{-\infty}^\infty {w_N(t) \dt \over x - t}
$$
or in other words, we have 
$$
-\pi \HH w_N(\mu_k) = V'(\mu_k)
$$

Formally (see a more detailed explanation below), $w_N(x)$ tends to a continuous limit (which we have guessed to be $w(x) = { \sqrt{2-x^2}  \over \pi}$).  We expect this limit to satisfy 

$$
 \HH w(x) = -{V'(x) \over \pi}
$$
Indeed:
    

In [155]:
x = Fun(-sqrt(2) .. sqrt(2))
w = sqrt(2-x^2)/π

norm(hilbert(w) + x/π)

0.0

Why is it $[-\sqrt 2, \sqrt 2]$? 

We thus want to choose the interval $[a,b]$ so that there exists a $w(x)$  satisfying

1. $w$ is bounded
2. $w(x) \geq 0$ for $a \leq x \leq b$
2. $\int_a^b w(x) \dx = 1$
3. $\HH_{[a,b]} w(x) = - x/\pi$

As we saw above, there exists a bounded solution to 
$\HH_{[-b,b]} u =  - x/\pi$,
namely  $u(x) = { \sqrt{b^2-x^2} \over \pi}$. The choice $b = \sqrt{2}$ ensures that $\int_{-b}^b u(x) \dx = 1$, hence $u(x) = w(x)$.

### Aside: Explanation of limit of $w_N(x)$

This is beyond the scope of the course, but the convergence of $w_N(x)$ to $w(x)$ is known as weak-\* convergence. A simple version of this is that
$$
   \int_c^d w_N(x) \dx \rightarrow \int_c^d w(x) \dx
$$
for every choice of interval $(c,d)$. $   \int_c^d w_N(x) \dx $ is precisely the number of charges in $(c,d)$ scaled by $1/N$,  which is exactly what a histogram plots.

In [247]:
a = -0.1; b= 0.3;
w = Fun(x -> sqrt(2-x^2)/π, a .. b)
sum(w)  # integral of w(x) between a and b

0.1790059168895313

In [248]:
length(filter(λ -> a ≤ λ ≤ b, λ(10.0)/sqrt(N)))/N # integral of w_N(x) between a and b 

0.18

Another varient of this is that the Cauchy transforms converge, that is, for $z$ on any  contour $\gamma$ surrounding $a$ and $b$ (now the support of $w$) we have
$$
\int_a^b {w_N(x) \over x - z} \dx \rightarrow \int_a^b {w(x) \over x - z} \dx
$$
converges uniformly with respect to $N \rightarrow \infty$.
here we demonstrate it converges at a point:

In [251]:
x = Fun(-sqrt(2) .. sqrt(2))
w = sqrt(2-x^2)/π
z = 0.1+0.2im
cauchy(w, z)

0.1949409042727309 + 0.013681505291622226im

In [255]:
(sum(1./((λ(10.0)/sqrt(N)) .- z))/N)/(2π*im)

0.19542122629792996 + 0.0137245440946901im